In [4]:
import pandas as pd
import numpy as np

path = "drive/MyDrive/Capstone/"
crop_1 = "Barley"
crop_2 = "Maize"
crop_6 = "Sorghum"
crop_7 = "Teff"
crop_8 = "Wheat"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
crop = crop_7
dfs = []

features = { "region": "Region", 
                "zone": "Zone", 
                "district": "District", 
                "production": "Production Of Condition in Kg",
                "area": "Area in Square meter", 
                "irrigated": 'Is Field Irrigated?',
                #"erosion": 'Is Field Prevented form Erosion',
                "seed": 'Seed / Seedling Type',
                "fertilizer" : 'Is Fertilizer Used?',
                "seed_qty": 'Quantity of indigenous seeds used',
                "fetilizer_type" : 'Type of fertilizer used if any?',
                "erosion_prevention" : 'Common way of prevention',
                "fertilizer_qty" : "Fertilizer Quantity",
                "damage": "Was crop damaged?",
                "previous": 'What was the previous state of the field?',
                #"damage_percent": 'Damage in Percent', 
                }

column_rename_dict = {}

column_rename_dict.update(dict.fromkeys(["DAMAGE" , "Was crop damaged ?", "Is there Damage?", "Any Damage?", "Any Damage", "Was crop dammaged"],
                             features["damage"]))

column_rename_dict.update(dict.fromkeys(['Used Irrigation?','Is field irrigated','Irrigation Used','Irrigation used?','IRRG'],
                             features["irrigated"]))

#column_rename_dict.update(dict.fromkeys(['Is the Soil Erosion', 'SERRO', 'Soil Erosion', 'Prevention for soil erosion?', 'Is Field Prevented form Erosion', ],
#                             features["erosion"]))

column_rename_dict.update(dict.fromkeys(['SEEDTYPE', 'Seed type', 'Seed Type', 'SEED TYPE',],
                             features["seed"]))

column_rename_dict.update(dict.fromkeys(['Is fertilizer used', 'Ferilizer', 'FERT', 'Fertilizer Used?', 'Fertilizer Used'],
                             features["fertilizer"]))

column_rename_dict.update(dict.fromkeys(['Weight of Indigenous seed', 'Weight of non-improved seed',  'Weight of Non-improved Seed', 'WTNISEED',],
                             features["seed_qty"]))

column_rename_dict.update(dict.fromkeys(['Fertilizer Type','FERTTYPE',],
                             features["fetilizer_type"]))

column_rename_dict.update(dict.fromkeys(['If field Prevented form Erosion, common way of prevention', 'MERRO', 'Measures taken to prevent Soil Erosion',],
                             features["erosion_prevention"]))

column_rename_dict.update(dict.fromkeys(['What was the previous state of the field ?', 'D26','What was the field used for?',],
                             features["previous"]))

start_year = 2010

for year in range(start_year, 2022):
  df_temp = pd.read_csv(f"{path}Clean_Meher_{year}_{crop}.csv")

  df_temp.rename(columns=column_rename_dict, inplace=True)

  dfs.append(df_temp)


In [7]:
columns = list(features.values())

for idx, df in enumerate(dfs):
  dfs[idx] = df[columns]

for df in dfs:
  print(df.shape)

(14431, 14)
(28395, 14)
(15450, 14)
(26433, 14)
(29902, 14)
(27553, 14)
(25637, 14)
(25404, 14)
(13892, 14)
(16243, 14)
(24682, 14)
(23597, 14)


In [8]:
df = pd.concat(dfs)
df.shape

(271619, 14)

In [9]:
min_threshold, max_threshold = df[[features["production"]]].quantile([0.01, 0.99]).values[:, 0]
min_threshold, max_threshold

(8.329079999999998, 1274.654480000002)

In [10]:
df = df[(df[features["production"]] > min_threshold) & (df[features["production"]] < max_threshold)]

In [11]:
df_region = pd.get_dummies(df[features["region"]]).iloc[:, :-1]
df_zone = pd.get_dummies(df[features["zone"]]).iloc[:, :-1]
df_district = pd.get_dummies(df[features["district"]]).iloc[:, :-1]
df_damage = pd.get_dummies(df[features["damage"]]).iloc[:, :-1]
df_irrigated = pd.get_dummies(df[features["irrigated"]]).iloc[:, :-1]
#df_erosion = pd.get_dummies(df[features["erosion"]]).iloc[:, :-1]
df_erosion_prevention = pd.get_dummies(df[features["erosion_prevention"]]).iloc[:, :-1]
df_seed = pd.get_dummies(df[features["seed"]]).iloc[:, :-1]
df_fertilizer = pd.get_dummies(df[features["fertilizer"]]).iloc[:, :-1]
df_fetilizer_type = pd.get_dummies(df[features["fetilizer_type"]]).iloc[:, :-1]
df_previous = pd.get_dummies(df[features["previous"]]).iloc[:, :-1]

In [12]:
df = df.drop([features["region"], features["zone"], features["district"], features["damage"], features["irrigated"], features["seed"], features["fertilizer"], features["erosion_prevention"], features["fetilizer_type"], features["previous"]], axis = 1)


In [13]:
df_save = df.iloc[ : 1]

for col in df_save.columns:
    df_save[col].values[:] = 0

df_save

,Production Of Condition in Kg,Area in Square meter,Quantity of indigenous seeds used,Fertilizer Quantity
0,0.0,0.0,0.0,0.0


In [14]:
df = pd.concat([df, df_region, df_zone, df_district, df_damage, df_irrigated, df_erosion_prevention, df_seed, df_fertilizer, df_fetilizer_type, df_previous  ], axis = 1)

In [15]:
X = df.loc[:, df.columns != 'Production Of Condition in Kg']
y = df.loc[:, df.columns == "Production Of Condition in Kg"]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [17]:
from sklearn.preprocessing import StandardScaler, Normalizer
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape, X_test.shape

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  FutureWarning,


((186329, 84), (79856, 84))

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import Input

model = Sequential()

In [19]:
input_layer = X_train.shape[1]
model.add(Input(shape=(input_layer,)))
model.add(Dense(units = input_layer / 2, kernel_initializer="uniform", activation = "relu"))
model.add(Dense(units = input_layer / 2, kernel_initializer="uniform", activation = "relu"))
model.add(Dense(units = input_layer / 2, kernel_initializer="uniform", activation = "relu"))
model.add(Dense(units = input_layer / 2, kernel_initializer="uniform", activation = "relu"))
model.add(Dense(units = 1, kernel_initializer ="uniform"))
model.compile(loss='mean_squared_error', optimizer='adam',  metrics=['RootMeanSquaredError'])

In [20]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size = 100, epochs = 100)

Epoch 1/100
1864/1864 [==============================] - 11s 5ms/step - loss: 26914.3867 - root_mean_squared_error: 164.0560 - val_loss: 20141.2285 - val_root_mean_squared_error: 141.9198
Epoch 2/100
1864/1864 [==============================] - 5s 3ms/step - loss: 19618.6113 - root_mean_squared_error: 140.0665 - val_loss: 19034.2246 - val_root_mean_squared_error: 137.9646
Epoch 3/100
1864/1864 [==============================] - 5s 3ms/step - loss: 19003.2246 - root_mean_squared_error: 137.8522 - val_loss: 18870.7617 - val_root_mean_squared_error: 137.3709
Epoch 4/100
1864/1864 [==============================] - 5s 3ms/step - loss: 18663.1289 - root_mean_squared_error: 136.6131 - val_loss: 18378.4297 - val_root_mean_squared_error: 135.5671
Epoch 5/100
1864/1864 [==============================] - 5s 3ms/step - loss: 18426.2715 - root_mean_squared_error: 135.7434 - val_loss: 18208.5234 - val_root_mean_squared_error: 134.9390
Epoch 6/100
1864/1864 [==============================] - 5s 3ms/

In [ ]:
def build_model(optimizer):
  model = Sequential()
  input_layer = X_train.shape[1] / 2
  model.add(Input(shape=(X_train.shape[1],)))
  model.add(Dense(units = input_layer, kernel_initializer = "uniform", activation = "relu"))
  model.add(Dense(units = input_layer, kernel_initializer="uniform", activation = "relu"))
  model.add(Dense(units = input_layer, kernel_initializer="uniform", activation = "relu"))
  model.add(Dense(units = input_layer, kernel_initializer="uniform", activation = "relu"))
  model.add(Dense(units = 1, kernel_initializer ="uniform"))
  model.compile(loss='mean_squared_error', optimizer=optimizer,  metrics=['RootMeanSquaredError'])
  return model

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

classifier = KerasRegressor(build_fn = build_model)
parameters = {'batch_size': [10, 50],
              'epochs': [100],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
grid_search.best_estimator_.model.summary()

In [ ]:
grid_search.best_estimator_.score(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)

In [22]:
from keras import backend as K

def coeff_determination(y_true, y_pred):

    y_true = K.constant(y_true)
    y_pred = K.constant(y_pred)
    
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [23]:
y_pred = model.predict(X_train)
coeff_determination(y_train.to_numpy(), y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.74852645>

In [24]:
y_pred = model.predict(X_test)
coeff_determination(y_test.to_numpy(), y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7301278>

In [ ]:
import pickle

filename = path + crop + "_Model"

pickle.dump(model, open(filename, "wb"))

model

In [26]:
filename = path + crop + "_Scaler"

pickle.dump(sc, open(filename, "wb"))

sc

StandardScaler()

In [27]:
filename = path + crop + "_Df"

pickle.dump(df_save, open(filename, "wb"))

df_save

,Production Of Condition in Kg,Area in Square meter,Quantity of indigenous seeds used,Fertilizer Quantity
0,0.0,0.0,0.0,0.0


In [28]:
dfs = [df_region, df_zone, df_district, df_damage, df_irrigated, df_erosion_prevention, df_seed, df_fertilizer, df_fetilizer_type, df_previous]
labels = ["Region", "Zone", "District", "Damage", "Irrigated", "Erosion", "Seed", "Fertilizer", "Fertilizer_Type", "Previous"]

for idx in range(0, len(dfs)):
  df_selected = dfs[idx].iloc[ : 1]

  for col in df_selected.columns:
      df_selected[col].values[:] = 0

  filename = path + crop + "_" + labels[idx]
  pickle.dump(df_selected, open(filename, "wb"))

  print( labels[idx])
  

Region
Zone
District
Damage
Irrigated
Erosion
Seed
Fertilizer
Fertilizer_Type
Previous
